
## Глава 3 Теоремы теории вероятностей

**Кандидат должен знать:**

- Закон больших чисел и усиленный закон больших чисел
- Центральная предельная теорема
- Многомерные распределения

**Материалы:**
- [Райгородский курс по тер. веру лекции 8,9,12,13](https://www.youtube.com/watch?v=n7jx2MfIeX4&list=PLthfp5exSWEr8tRK-Yf-i9aXgcFJ-O16d&index=8)
- [курс ТВ  на степике из частей 2,4](https://stepik.org/course/3089)


## Задачи с кодингом


### 1. Генерация нормальной случайной величины

**Дано:**

Пусть у нас есть случайная величина $\xi \sim U[0,1]$ . 

**Задача:**
1. На основе данной случайной величины с помощью ЦПТ осуществите моделирование  ($n = 10000$) случайной величины  $\eta \sim N(\mu, \sigma^2)$

2. Постройте график полтности распределения и сравните с теоритическим. 

3. Вычислить математического ожидания и дисперсии, сравнить их с истинными значениями.

**Примечания**

1. Для генерации реализаций $\xi$ можно восппользоваться функцией numpy.random.random()
2.  Для построения теоритического графика плотности распределения нормальной величины можно воспользоваться функцией scipy.stats.norm.ppf

In [2]:
import math
import numpy as np
import plotly.express as px
from scipy.stats import norm

In [27]:
n = 10**4
distribution = np.array([np.random.random(4).mean() for _ in range(n//4)])

In [28]:
fig = px.histogram(distribution, nbins=50, title='Distribution of Array', labels={'value':'Value'}, marginal="rug", height=700,  width=1000)
fig.show()

In [31]:
pdf_values = norm.pdf(distribution)
fig = px.line(x=distribution, y=pdf_values, title='Probability Density Function', labels={'x':'Value', 'y':'PDF'})
fig.show()

In [30]:
mean = distribution.mean()
variance = distribution.var()
print(f'Mean: {mean}, Variance: {variance}')

Mean: 0.5022374886882263, Variance: 0.021656398197903774


### 2. Казино 

В казино имеется две игры:

**a. Кубики:**

Игрок платит 4&#36; и бросает два кубика. Если в сумме выпало 8 и более то игрок получает приз равный полученной сумме.

**b. Разгадай код:**
    Есть пятизначный код (на каждой позиции стоит цифра от 0 до 9).  Игрок платит 10&#36; и пытается угадать код:
    
    - Если игрок меньше двух позиций то ничего не получает 
    - Если игрок угадал ровно две позиции из 5-ти, то он получает 50
    - Если игроку гадал ровно три позиции из 5-ти, то выигрыш 200
    - Если игроку гадал ровно четыре позиции из 5-ти, то выигрыш 5000
    - Если игрок угадал весь код правильно то выигрыш 100000.
    После каждой попытка код меняется.

**Задача:**

Пусть $\xi ,  \eta $ - случайные величины равные выигрышу(проигрышу) игрока в раунде.
1. Посчитать $E\xi$ и $E\eta$
2. Смоделировать случайные величины $\xi$ и $\eta $
3. Построить графики сходимости среднего выигрыша(проигрыша) игрока к математическому ожиданию.
4. Одинаковая ли скорость сходимости  сл. величины к своему математическому ожиданию. Почему?
4. Какое бы $n$ - количество эеспериментов вы би взяли, для оценки мат. ожидания?




In [25]:
win_amount = np.array([0 if i < 8 else i for i in range(0, 13)]) - 4

possible_dice_values = [i+j for i in range(1, 7) for j in range(1, 7)]
count_values = [possible_dice_values.count(i) for i in range(2, 13)]
probs = list(map(lambda x: x/36, count_values))

E_xi = sum([prob*value for prob, value in zip(probs, win_amount[2:])])

print("probs: " , ",  ".join([f"{i+2}: {count_values[i]}/36" for i in range(11)]))
print("probs sum" , sum(count_values)/36)
print("E_xi" , round(E_xi, 5))

probs:  2: 1/36,  3: 2/36,  4: 3/36,  5: 4/36,  6: 5/36,  7: 6/36,  8: 5/36,  9: 4/36,  10: 3/36,  11: 2/36,  12: 1/36
probs sum 1.0
E_xi -0.11111


In [18]:
dice_values = np.random.randint(1, 7, 10000000)
dice_sums = dice_values[::2] + dice_values[1::2]
xi_values = win_amount[dice_sums]

print("E", xi_values.mean())

E -0.1099578


b. Разгадай код:

In [22]:
p = 0.1
q = 0.9
guess_code_win_amounts = np.array([0, 0, 50, 200, 5000, 100000]) - 10
code_game_probs = [math.comb(5, i) * p**i * q**(5-i) for i in range(6)]
E_eta = sum([prob*value for prob, value in zip(code_game_probs, guess_code_win_amounts)])

print("probs" , list(map(lambda x: round(x, 5), code_game_probs)))
print("probs sum" , sum(code_game_probs))
print("E_eta" , round(E_eta, 5))

probs [0.59049, 0.32805, 0.0729, 0.0081, 0.00045, 1e-05]
probs sum 1.0
E_eta -1.485


In [23]:
np.random.seed(113)
N_plays = 4*10**7

zeros_code = np.array([0, 0, 0, 0, 0])
arrange_code = np.array([1, 2, 3, 4, 5])
random_values_code = np.random.randint(0, 10, (N_plays, 5))

code_guesses = np.random.randint(0, 10, (N_plays, 5))

is_guess_zeros_code = code_guesses[:] == zeros_code 
zeros_guesses = is_guess_zeros_code.sum(axis=1)
zeros_guesses_win_amounts = guess_code_win_amounts[zeros_guesses]
print("zeros E", zeros_guesses_win_amounts.mean())

is_guess_arrange_code = code_guesses[:] == arrange_code 
arrange_guesses = is_guess_arrange_code.sum(axis=1)
arrange_guesses_win_amounts = guess_code_win_amounts[arrange_guesses]
print("arrange E", arrange_guesses_win_amounts.mean())

is_guess_random_code = code_guesses == random_values_code
random_guesses = is_guess_random_code.sum(axis=1)
random_guesses_win_amounts = guess_code_win_amounts[random_guesses]
print("random E", random_guesses_win_amounts.mean())

print("real E", round(E_eta, 5))

zeros E -1.49063625
arrange E -1.468605
random E -1.48721875
real E -1.485


### Случайный граф


**Условие**
Пусть G(n,p) -  случайный граф на $n$ вершинах, где $p$ - это вероятность появления ребра между любыми двумя вершинами.

Обозначим за $\xi(n,p)$ - количество треугольников (т.е. полных подграфов на трёх вершинах) в случайном графе  

**Задача 1:**

1. Найти $E\xi$
2. Реализовать моделирование случайного случайного графа и функцию нахождения количества треугольников. 
3. Рассмотрим G(3,p) - случайный граф на трёх вершинах. Реализуйте $\tau$ - случайная величина равная 1 если граф полный и 0 иначе.


**Задача 2:**

    Пусть M(k) - среднее количество треугольников после $k$ генераций случайного графа. 
   
   И пусть T(k)  - среднее количество треугольников после генераций случайного графа на трёх вершинах (т.е. на каждой итерации генерируем $C_n^3$ графов на трёх вершинах).

1. Верно ли что $M(k) \rightarrow E\xi$ при $k \rightarrow \infty$ 
2. Верно ли что $T(k) \rightarrow  E\xi$ при $k \rightarrow \infty$

3. Рассмотрите два графа $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$.  Построите графики  $M(k)$, $T(k)$ и $E\xi$ (константа) для заданных графов.

**Задача 3**:

Проделайте аналогичные рассуждения для $D \xi$ т.е.
- Найдите  $D \xi$  математически.
- Посчитайте дисперсию количества треугольников, k раз реализуя случайный граф.
- Посчитайте дисперсию количества треугольников, k раз генерируя  $C_n^3$ реализаций случайной величины $\tau$ .
- оцените сходимость
- Постройте графики для графов  $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$.